<a href="https://colab.research.google.com/github/STCole184/earthquakeDamage/blob/master/EarthquakeDamage_geo1geo2categoricalXGBRegressorCustomThresholds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is my solution to the DrivenData competition Richter's Predictor: Modeling Earthquake Damage. As of the writing of this notebook, my solution is ranked 389 of 2290 solutions, i.e. it is in the 83rd percentile.

I tried a number of solutions on my way to this solution. A summary of my experiments and their results are below.

My best solution ended up using all features except for the geo_level_3_id feature. This feature had too many values to efficiently one-hot encode and use in training my algorithms. It also used regression instead of categorization. The problem is one of ordinal classification, and so the regression model was appropriate. I then took the output of the regression model and manually found optimal thresholds between damage_levels 1 and 2, and 2 and 3. These thresholds were not at the 0.5 mark, and so this manual discovery of thresholds added value to the model.

As you can see below, I tried using class rebalancing techniques such as SMOTE and random undersampling of the dominant class of data since the classes of the data were fairly imbalanced. I did not find these techniques to improve my model however.

I found my model to over predict the dominant class (damage_level 2) in all of my experiments. Setting thresholds for regression scores between the classes helped to ameliorate this deficiency, but the deficiency remains in my model with recall scores for the minority classes at 47% and 59%. Even breaking the problem down into two binary classifiers to differentiate between damage_level 1 and the rest of the data points, and damage_level 3 and the rest of the data points did not improve my model's performance on this front, or at all.




---



---



**Experiments:**

Baseline (random forest classifier, no one-hot encoding of geo features which are technically numeric, even though those numbers are just codes and have no numeric meaning) F1 score: 0.5749697818537634; Conditions: all features, geo features not categorical

RandomForest classifier with only geo_1 one-hot encoded, F1 score: 0.626196734521594; Conditions: did not use geo_2 or geo_3 but made geo_1 categorical; Issues: predictions highly skewed to class 2

GradientBoost classifier with only geo_1: 0.684; Conditions: did not use geo_2 or geo_3 but made geo_1 categorical, used GradientBoost instead of RandomForest; Issues: predictions still skewed to class 2 but not as much

XGBoostClassifier with geo_1 and geo_2: 0.674; Conditions: did not use geo_3 but made geo_1 and geo_2 categorical, used XGBoost instead of RandomForest; Issues: predictions still skewed to class 2 but not as much

XGBoostRegressor with geo_1 and geo_2: 0.714; Conditions: did not use geo_3 but made geo_1 and geo_2 categorical, used XGBoost instead of RandomForest; Issues: predictions still skewed to class 2 but not as much

XGBoostRegressor with geo_1 and geo_2 and custom thresholds: 0.7175; Conditions: did not use geo_3 but made geo_1 and geo_2 categorical, used XGBoost instead of RandomForest; Issues: predictions still skewed to class 2 but not as much

XGBoostRegressor with geo_1 and geo_2 and custom thresholds and smote for balancing training classes: 0.695; Conditions: did not use geo_3 but made geo_1 and geo_2 categorical, used XGBoost instead of RandomForest; Issues: predictions still skewed to class 2 but not as much

XGBoostRegressor with geo_1 and geo_2 and custom thresholds and random undersampling for balancing training classes: 0.698; Conditions: did not use geo_3 but made geo_1 and geo_2 categorical, used XGBoost instead of RandomForest; Issues: predictions still skewed to class 2 but not as much

Two binary XGBoostClassifiers with geo_1 and geo_2 and custom thresholds to make the classification task easier by splitting it into two classifiers (one to identify just damage_level 1 and one to identify just damage_level 2): 0.674; Conditions: did not use geo_3 but made geo_1 and geo_2 categorical, used XGBoost instead of RandomForest; Issues: I could not get the individual classifier to perform better than the single classifier for all categories.

In [0]:
# Prepare matplotlib for use in notebook
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [0]:
# Import other needed libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from xgboost import XGBRegressor
from sklearn.decomposition import PCA

In [0]:
# Mount gdrive to access input files
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
# Check folders in my working directory to see how to build paths to input files
!ls

adc.json  drive  sample_data


In [0]:
#Read in data

train_vals = pd.read_csv("drive/My Drive/Earthquake Damage/train_values.csv")
train_labs = pd.read_csv("drive/My Drive/Earthquake Damage/train_labels.csv")
test_vals = pd.read_csv("drive/My Drive/Earthquake Damage/test_values.csv")
submission_format = pd.read_csv("drive/My Drive/Earthquake Damage/submission_format.csv")

In [26]:
# Now that we have our data loaded, first let's make sure the train data and the train labels are aligned by building id.
# The lengths of matches between the two dataframes on building_id and the lengths of each dataframe independently are all the same.
# This means they are aligned.

print(sum(train_vals['building_id'] == train_labs['building_id']))

print(train_vals.shape)

print(train_labs.shape)

260601
(260601, 39)
(260601, 2)


In [8]:
# Check training data shape and content to ensure it is as expected
print(train_vals.shape)
train_vals.head()

(260601, 39)


,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [10]:
print(train_labs.shape)
train_labs.head()

(260601, 2)


,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [0]:
# Check for missing values in the training data. Fortunately, there aren't any!

train_vals.isnull().sum()

building_id                               0
geo_level_1_id                            0
geo_level_2_id                            0
geo_level_3_id                            0
count_floors_pre_eq                       0
age                                       0
area_percentage                           0
height_percentage                         0
land_surface_condition                    0
foundation_type                           0
roof_type                                 0
ground_floor_type                         0
other_floor_type                          0
position                                  0
plan_configuration                        0
has_superstructure_adobe_mud              0
has_superstructure_mud_mortar_stone       0
has_superstructure_stone_flag             0
has_superstructure_cement_mortar_stone    0
has_superstructure_mud_mortar_brick       0
has_superstructure_cement_mortar_brick    0
has_superstructure_timber                 0
has_superstructure_bamboo       

In [13]:
# Checking to see how many unique values every feature has, especially the categorical features. Feature geo_level_3_id has a lot of unique values
train_vals.apply(lambda x: len(x.unique()), axis=0)

building_id                               260601
geo_level_1_id                                31
geo_level_2_id                              1414
geo_level_3_id                             11595
count_floors_pre_eq                            9
age                                           42
area_percentage                               84
height_percentage                             27
land_surface_condition                         3
foundation_type                                5
roof_type                                      3
ground_floor_type                              5
other_floor_type                               4
position                                       4
plan_configuration                            10
has_superstructure_adobe_mud                   2
has_superstructure_mud_mortar_stone            2
has_superstructure_stone_flag                  2
has_superstructure_cement_mortar_stone         2
has_superstructure_mud_mortar_brick            2
has_superstructure_c

In [0]:
#Let's eliminate the building_id column since it won't be used as a feature. Let's also eliminate the geo_level_3_id variable.
# The geo_level_3_id variable has so many unique values, one-hot encoding it could cause memory issues (and after trying it, I found
# that it does.

In [14]:
train_vals.drop(['building_id', 'geo_level_3_id'], axis=1, inplace=False).columns

Index(['geo_level_1_id', 'geo_level_2_id', 'count_floors_pre_eq', 'age',
       'area_percentage', 'height_percentage', 'land_surface_condition',
       'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type',
       'position', 'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_heal

In [0]:
# Creating a "features" dataframe to use for training
features_to_drop = ['building_id', 'geo_level_3_id']
train_features = train_vals.drop(features_to_drop, axis=1, inplace=False)

In [0]:
#I'm going to add the test features to the train features and one-hot encode them at the same time to make sure that all of the 
# values in the test features are represented in the one-hot encoder

test_features = test_vals.drop(features_to_drop, axis=1, inplace=False)

combined_features = pd.concat([train_features, test_features])

In [0]:
# Let's note the length of train_features so that we know where to split the transformed set of features
train_length = train_features.shape[0]

In [0]:
# Fit the one-hot encoder for all categorical variables except for geo_level_3_id because of the memory issue mentioned above

transformer_name = 'ohe_on_all_categorical_features'
transformer = OneHotEncoder(sparse=False)
columns_to_encode = ['geo_level_1_id', 'geo_level_2_id', 'land_surface_condition', 'foundation_type', 'roof_type', 
                     'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 
                     'legal_ownership_status']

ohe_final = ColumnTransformer([
    (transformer_name, transformer, columns_to_encode)], 
    remainder='passthrough')

ohe_final.fit(combined_features);

In [21]:
# Transform the combined features using the one-hot encoder. Check shape of transformed features.

combined_features_transformed = ohe_final.transform(combined_features)
print('Shape of transformed data matrix: ', combined_features_transformed.shape)

Shape of transformed data matrix:  (347469, 1514)


In [0]:
# Subset the transformed features for just the training data

train_features_transformed_df = pd.DataFrame(combined_features_transformed[0:260601, ])

In [23]:
# Check the transformed training features to ensure their shape and content are as expected. Nothing seems out of place

train_features_transformed_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,30.0,6.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0,5.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0,6.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,30.0,8.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# Check the shape of the transformed training features to ensure they align to expectations. They do: their length is the same as the
# original training dataframe and the width is the same as the transformed features dataframe

train_features_transformed_df.shape

(260601, 1514)

In [24]:
# Subset the transformed test features and check their shape. Their shape looks good.

test_features_transformed_df = pd.DataFrame(combined_features_transformed[260601:, ])

test_features_transformed_df.shape

(86868, 1514)

In [0]:
# During experimentation, I took a random sample of training data to work with in order to shorten the cycle time of my model trainings.
# We don't need this block of code for the final solution, but I thought it was worth mentioning that I did this when creating the solution.

'''# Let's take a random sample of the data to speed up the training-feedback cycle.

train_features_transformed_df_labs = pd.concat([train_features_transformed_df, train_labs['damage_grade']], axis=1)
train_features_transformed_df_labs.shape

train_features_transformed_df_sample = train_features_transformed_df_labs.sample(frac=.01, random_state=42)
train_features_transformed_df_sample.shape

train_labs_sample = train_features_transformed_df_sample['damage_grade']
train_features_transformed_df_sample.drop(['damage_grade'], axis=1, inplace=True)
print(train_labs_sample.shape)
print(train_features_transformed_df_sample.shape)'''

(2606,)
(2606, 1514)


In [0]:
# Now let's split our transformed training features and the corresponding labels into a train and test set.

#X_train, X_test, y_train, y_test = train_test_split(train_features_transformed_df_sample, train_labs_sample, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(train_features_transformed_df, train_labs['damage_grade'], test_size=0.2, random_state=42)

In [30]:
# Let's check to see that the split worked and we have 80% of the training features in the train set and 20% in the test set

print("Shape of the complete data set:", train_features_transformed_df.shape)
print("Shape of the train data set:", X_train.shape)
print("Shape of the test data set:", X_test.shape)

print("\n\nShape of the complete data labels:", train_labs['damage_grade'].shape)
print("Shape of the train data labels:", y_train.shape)
print("Shape of the test data labels:", y_test.shape)

Shape of the complete data set: (260601, 1514)
Shape of the train data set: (208480, 1514)
Shape of the test data set: (52121, 1514)


Shape of the complete data labels: (260601,)
Shape of the train data labels: (208480,)
Shape of the test data labels: (52121,)


In [0]:
# Let's now build some functions to make it easier to evaluate the performance of the models we build.
# My best solution used a single XGBRegressor to make continuous predictions from 1 to 3 for individual data points.
# I then take those regression scores and manually determine the optimal thresholds between the three categories of damage: 1, 2, 3.
# My model evaluation function reflects this process.
# The specific performance metric for the model was given by the competition, and it's the F1 score.

def model_evaluation(model, X, y_true):
    y_labels = model.predict(X)
    y_labels_df = pd.DataFrame(y_labels)
    y_labels_df['damage_grade'] = 2
    y_labels_df.loc[y_labels_df[0] < 1.6, 'damage_grade'] = 1
    y_labels_df.loc[y_labels_df[0] > 2.46, 'damage_grade'] = 3
    y_labels_df['count'] = 1
    predictions_pivot = y_labels_df.groupby(['damage_grade']).count()
    predictions_pivot['percentage'] = predictions_pivot['count']/predictions_pivot['count'].sum()
    scores = {}
    scores['f1_score'] = f1_score(y_true, y_labels_df['damage_grade'], average='micro')
    return scores, predictions_pivot, y_labels_df['damage_grade']

def print_model_evaluation(model_name, scores, pivot):
    print('{} evaluation \n'.format(model_name))
    for metric, score in scores.items():
        print('Test {}: {}'.format(metric, score))
        print(pivot)

In [0]:
# Before training my first model, I know that the proportions of the damage categories in the training data will be an important guide in
# determining the overall performance of the model. I create a function to give the proportion of damage categories in any set of damage labels
# below.

def createClassPivot(labels):
  
  labels_df = pd.DataFrame(labels)
  labels_df['count'] = 1
  labels_pivot = labels_df.groupby(['damage_grade']).count()
  labels_pivot['percentage'] = labels_pivot['count']/labels_pivot['count'].sum()

  return labels_pivot

In [34]:
# Below we can see that the three classes are skewed towards damage_grade 2 (56% of data points), followed by damage_grade 3 (33%)
# and finally damage_grade 1 (9%). We will need to check that the output of our model is performing high on its F1 score, but that
# it also conforms roughly to the proportions below.

train_labs_test = createClassPivot(train_labs)
train_labs_test

,building_id,count,percentage
damage_grade,,,
1,25124,25124,0.096408
2,148259,148259,0.568912
3,87218,87218,0.334680


In [35]:
# Here we create our training pipeline with an XGBRegressor and a gridsearcher. I've already input the optimal hyperparameters for the model

classifier_pipe = Pipeline([('gb', XGBRegressor(random_state=42, objective='reg:squarederror', n_estimators=100 , booster='gbtree', reg_lambda=1, reg_alpha=0))])

classifier_gs = GridSearchCV(classifier_pipe, 
                     cv=5, 
                     param_grid={'gb__max_depth': [6],
                                  'gb__learning_rate': [0.3],
                                 'gb__tree_method': ['hist']
                    }
                    )
classifier_gs.fit(X_train, y_train)


# I print the model's performance on the training set to see how well I am fitting the model to the train data

classifier_gs_scores_train, pivot_train, model_labels_train = model_evaluation(classifier_gs, X_train, y_train)
print_model_evaluation('XGBoost', classifier_gs_scores_train, pivot_train)

# I print the model's performance on the test set to see how well it generalizes to new data. The balance between train set performance
# and test set performance shows whether I am over- or underfitting the model

classifier_gs_scores_test, pivot_test, model_labels_test = model_evaluation(classifier_gs, X_test, y_test)
print_model_evaluation('XGBoost', classifier_gs_scores_test, pivot_test)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/x

XGBoost evaluation 

Test f1_score: 0.7259401381427475
                   0   count  percentage
damage_grade                            
1              14636   14636    0.070203
2             137621  137621    0.660116
3              56223   56223    0.269681
XGBoost evaluation 

Test f1_score: 0.7175035014677386
                  0  count  percentage
damage_grade                          
1              3672   3672    0.070451
2             34507  34507    0.662056
3             13942  13942    0.267493


In [0]:
# The results above do not indicate overfitting, which is good. As mentioned before, these are my best scores in any case.
# The results of my model however do show that my model is producing predictions in proportions that are out of balance with the
# proportions of predictions in the training data. My predictions for damage_grade 1 and 3 are too low. The model is over-predicting
# damage_grade 2.

In [0]:
# Now let's look at a confusion matrix of my predictions and the actual damage_grade labels to see where my model is weakest

conf_m = pd.DataFrame(confusion_matrix(y_test, model_labels_test),
             index = ['class 1 actual', 'class 2 actual', 'class 3 actual'],
             columns = ['class 1 pred', 'class 2 pred', 'class 3 pred'])

In [38]:
conf_m

,class 1 pred,class 2 pred,class 3 pred
class 1 actual,2406,2726,38
class 2 actual,1224,24675,3588
class 3 actual,42,7106,10316


In [0]:
# It will help to divide the values in the confusion matrix by the totals for my class predictions to see the precision of my scores.
# Precision ranges from 66% to 74%, which isn't horrible given that the winning score for this competition is ~75%. It is weakest for 
# damage_grade 1 (65%).

print('Precision')
print(conf_m/conf_m.sum(axis=0))

Precision
                class 1 pred  class 2 pred  class 3 pred
class 1 actual      0.655229      0.078998      0.002726
class 2 actual      0.333333      0.715072      0.257352
class 3 actual      0.011438      0.205929      0.739923


In [0]:
# It will also help to divide the values in the confusion matrix by the total count for each of the true class scores to see the recall of my scores.
# Recall ranges from 47% to 83%, which isn't very good. It is weakest for damage_grade 1 (47%) but not much better for damage_grade 3 (59%).
# Recall is high for damage_grade 2 (83%), but all of this just shows what I could tell above: the model is over-predicting damage_grade 2.

print('Recall')
print(conf_m.transpose()/conf_m.sum(axis=1))

Recall
              class 1 actual  class 2 actual  class 3 actual
class 1 pred        0.465377        0.041510        0.002405
class 2 pred        0.527273        0.836809        0.406894
class 3 pred        0.007350        0.121681        0.590701


In [0]:
# My hope in manually identifying cutoff thresholds between the three damage_grades was to be able to force the damage_grade proportions
# of my model's output to conform better to the damage_grade proportions in the overall training data. This technique had limited effect.
# I give a few examples of different thresholds and their F1 scores and output class proportions below to illustrate.

In [39]:
# As a baseline, let's see the performance of the model when setting the threholds at the .5 mark. The F1 scores for both train and test
# and the class proportions of the outputs are all marginally worse.

def model_evaluation_find_threshold(model, X, y_true):
    y_labels = model.predict(X)
    y_labels_df = pd.DataFrame(y_labels)
    y_labels_df['damage_grade'] = 2
    y_labels_df.loc[y_labels_df[0] < 1.5, 'damage_grade'] = 1
    y_labels_df.loc[y_labels_df[0] > 2.5, 'damage_grade'] = 3
    y_labels_df['count'] = 1
    predictions_pivot = y_labels_df.groupby(['damage_grade']).count()
    predictions_pivot['percentage'] = predictions_pivot['count']/predictions_pivot['count'].sum()
    scores = {}
    scores['f1_score'] = f1_score(y_true, y_labels_df['damage_grade'], average='micro')
    return scores, predictions_pivot, y_labels_df['damage_grade']

def print_model_evaluation(model_name, scores, pivot):
    print('{} evaluation \n'.format(model_name))
    for metric, score in scores.items():
        print('Test {}: {}'.format(metric, score))
        print(pivot)

classifier_gs_scores_train, pivot_train, model_labels_train = model_evaluation_find_threshold(classifier_gs, X_train, y_train)
print_model_evaluation('XGBoost', classifier_gs_scores_train, pivot_train)

classifier_gs_scores_test, pivot_test, model_labels_test = model_evaluation_find_threshold(classifier_gs, X_test, y_test)
print_model_evaluation('XGBoost', classifier_gs_scores_test, pivot_test)

XGBoost evaluation 

Test f1_score: 0.7238440138142749
                   0   count  percentage
damage_grade                            
1              10388   10388    0.049827
2             147087  147087    0.705521
3              51005   51005    0.244652
XGBoost evaluation 

Test f1_score: 0.7143186047850195
                  0  count  percentage
damage_grade                          
1              2592   2592    0.049730
2             36858  36858    0.707162
3             12671  12671    0.243107


In [40]:
# Now let's see what the perforamnce measures look like when we set the thresholds to favor damage_grades 1 and 3 even more than my
# thresholds above (1.6, 2.46). The F1 scores for train and test are marginally worse, even though the class output proportions are
# marginally better. This tells us two things: 1) because we are optimizing fof F1 score, we should stick with our thresholds of (1.6, 2.46).
# 2) The regression algorithm is unfortunately not doing a great job of differentiating between damage_grade 1 and damage_grade 2 on the
# one hand, and on damage_grade 2 and damage_grade 3 on the other.

def model_evaluation_find_threshold(model, X, y_true):
    y_labels = model.predict(X)
    y_labels_df = pd.DataFrame(y_labels)
    y_labels_df['damage_grade'] = 2
    y_labels_df.loc[y_labels_df[0] < 1.61, 'damage_grade'] = 1
    y_labels_df.loc[y_labels_df[0] > 2.45, 'damage_grade'] = 3
    y_labels_df['count'] = 1
    predictions_pivot = y_labels_df.groupby(['damage_grade']).count()
    predictions_pivot['percentage'] = predictions_pivot['count']/predictions_pivot['count'].sum()
    scores = {}
    scores['f1_score'] = f1_score(y_true, y_labels_df['damage_grade'], average='micro')
    return scores, predictions_pivot, y_labels_df['damage_grade']

def print_model_evaluation(model_name, scores, pivot):
    print('{} evaluation \n'.format(model_name))
    for metric, score in scores.items():
        print('Test {}: {}'.format(metric, score))
        print(pivot)

classifier_gs_scores_train, pivot_train, model_labels_train = model_evaluation_find_threshold(classifier_gs, X_train, y_train)
print_model_evaluation('XGBoost', classifier_gs_scores_train, pivot_train)

classifier_gs_scores_test, pivot_test, model_labels_test = model_evaluation_find_threshold(classifier_gs, X_test, y_test)
print_model_evaluation('XGBoost', classifier_gs_scores_test, pivot_test)

XGBoost evaluation 

Test f1_score: 0.7259113584036837
                   0   count  percentage
damage_grade                            
1              15223   15223    0.073019
2             135660  135660    0.650710
3              57597   57597    0.276271
XGBoost evaluation 

Test f1_score: 0.716793614857735
                  0  count  percentage
damage_grade                          
1              3815   3815    0.073195
2             34000  34000    0.652328
3             14306  14306    0.274477


In [0]:
# Now, let's get predictions for the test set of data to submit in the competition. First we define a function to get the predictions
# using the thresholds I discovered in my experimentation.

def getPredictionAndAnalysis(X_features, model):

  y_labels = model.predict(X_features)
  y_labels_df = pd.DataFrame(y_labels)
  y_labels_df['damage_grade'] = 2
  y_labels_df.loc[y_labels_df[0] < 1.6, 'damage_grade'] = 1
  y_labels_df.loc[y_labels_df[0] > 2.46, 'damage_grade'] = 3
  y_labels_df['count'] = 1
  predictions_pivot = y_labels_df.groupby(['damage_grade']).count()
  predictions_pivot['percentage'] = predictions_pivot['count']/predictions_pivot['count'].sum()

  return y_labels_df['damage_grade'], predictions_pivot, y_labels_df

In [0]:
# Then we get the prediction output.
labels, test_pivot, test_pivot_df = getPredictionAndAnalysis(test_features_transformed_df, classifier_gs)

In [43]:
# Check the output
test_pivot_df

,0,damage_grade,count
0,2.854345,3,1
1,2.211867,2,1
2,2.049302,2,1
3,1.178822,1,1
4,2.859063,3,1
...,...,...,...
86863,2.140169,2,1
86864,2.408236,2,1
86865,2.116575,2,1
86866,2.209028,2,1


In [0]:
# Format the output in the submission format
test_labels_df = pd.DataFrame(test_pivot_df['damage_grade'])
test_labels_df['building_id'] = test_vals['building_id']
test_labels_df.columns = ['damage_grade', 'building_id']
test_labels_df = test_labels_df[['building_id', 'damage_grade']]

In [45]:
# Check it
test_labels_df.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3


In [46]:
# Compare it to the example submission format
submission_format.head()

,building_id,damage_grade
0,300051,1
1,99355,1
2,890251,1
3,745817,1
4,421793,1


In [0]:
# And print it out to my gdrive
uploaded = drive.CreateFile({'title': 'submission4.csv'})
uploaded.SetContentString(test_labels_df.to_string(index=False, ))
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1r5MXUa6D2y4GtKwgZKgqHeu0s-D_73IL
